In [3]:
!pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   --- ------------------------------------ 10.2/129.1 kB ? eta -:--:--
   --------- ----------------------------- 30.7/129.1 kB 435.7 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/129.1 kB 653.6 kB/s eta 0:00:01
   ------------------------------------ - 122.9/129.1 kB 722.1 kB/s eta 0:00:01
   -------------------------------------- 129.1/129.1 kB 691.3 kB/s eta 0:00:00


In [4]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from keras_tuner import Hyperband

In [5]:
# Load data from JSON file
file_path = 'C:/Users/HP/Desktop/angles_output_3.json'
data = []
with open(file_path) as f:
    for line in f:
        data.append(json.loads(line))

In [6]:
# Convert JSON data to DataFrame
df = pd.DataFrame(data)

In [7]:
# Separate features and labels
X = df.drop('Label', axis=1).values
y = df['Label'].values

In [8]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [9]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [10]:
# Reshape data for LSTM
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [11]:
# Function to build the model
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=128, step=16)))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [12]:
# Hyperparameter tuner
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='hyperparam_opt',
    project_name='lstm_verification_model'
)

In [13]:
# Perform hyperparameter search
tuner.search(X_train_lstm, y_train, epochs=50, validation_data=(X_test_lstm, y_test))

Trial 30 Complete [00h 00m 04s]
val_accuracy: 0.9470899701118469

Best val_accuracy So Far: 0.9523809552192688
Total elapsed time: 00h 01m 15s


In [14]:
# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best units: {best_hps.get('units')}")
print(f"Best dropout rate: {best_hps.get('dropout')}")
print(f"Best optimizer: {best_hps.get('optimizer')}")

Best units: 112
Best dropout rate: 0.30000000000000004
Best optimizer: adam


In [17]:
# Build the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_lstm, y_train, epochs=50, validation_data=(X_test_lstm, y_test))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3826 - loss: 1.4437 - val_accuracy: 0.7989 - val_loss: 1.0185
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7259 - loss: 0.9651 - val_accuracy: 0.8730 - val_loss: 0.8234
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8469 - loss: 0.7815 - val_accuracy: 0.8942 - val_loss: 0.6603
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8881 - loss: 0.6454 - val_accuracy: 0.8942 - val_loss: 0.5547
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9120 - loss: 0.5612 - val_accuracy: 0.8995 - val_loss: 0.4731
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9275 - loss: 0.4743 - val_accuracy: 0.9312 - val_loss: 0.4122
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9362 - loss: 0.4129 - val_accuracy: 0.9101 - val_loss: 0.3630
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9399 - loss: 0.3439 - val_accuracy: 0.9312 - val_loss

In [18]:
# Save the best model
best_model.save('best_verification_model.h5')

In [19]:
# Save the training history
import json
with open('best_model_history.json', 'w') as f:
    json.dump(history.history, f)